In [ ]:
#Import libraries & Extensions
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import panel as pn
import panel.widgets as pnw
import folium
import numpy as np
pn.extension("plotly", # for plotly
             "tabulator", # for displaying dataframes
            #  sizing_mode="stretch_width" # for folium
             )

In [ ]:
#Data set: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data/data?select=GlobalLandTemperaturesByCity.csv
GlobalTemperatures = pd.read_csv('/content/GlobalTemperatures.csv')
GlobalLandTemperaturesByCountry = pd.read_csv('/content/GlobalLandTemperaturesByCountry.csv')
GlobalLandTemperaturesByMajorCity = pd.read_csv('/content/GlobalLandTemperaturesByMajorCity.csv')

In [ ]:
#Function to change temperature from celsius to fahrenheit
def to_fahrenheit(celsius):
  fahrenheit = celsius * (9/5) + 32
  return fahrenheit

In [ ]:
#Drop NaN values & Change Dataframe temperature from Celsius to Fahrenheit
GlobalLandTemperaturesByCountry.dropna(subset=['AverageTemperature'], inplace=True)

fahrenheit = GlobalLandTemperaturesByCountry.AverageTemperature.map(to_fahrenheit)
GlobalLandTemperaturesByCountry['AverageTemperature'] = GlobalLandTemperaturesByCountry.AverageTemperature.apply(to_fahrenheit)
fahrenheit = GlobalLandTemperaturesByCountry.	AverageTemperatureUncertainty.map(to_fahrenheit)
GlobalLandTemperaturesByCountry['AverageTemperatureUncertainty'] = GlobalLandTemperaturesByCountry.AverageTemperatureUncertainty.apply(to_fahrenheit)

GlobalLandTemperaturesByCountry.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,39.8912,36.1292,Åland
5,1744-04-01,34.7540,40.4240,Åland
6,1744-05-01,44.0636,35.2202,Åland
7,1744-06-01,52.8962,34.8386,Åland
8,1744-07-01,59.6156,34.5380,Åland


In [ ]:
#Change Dataframe temperature from Celsius to Fahrenheit
GlobalLandTemperaturesByMajorCity.dropna(subset=['AverageTemperature'], inplace=True)

fahrenheit = GlobalLandTemperaturesByMajorCity.AverageTemperature.map(to_fahrenheit)
GlobalLandTemperaturesByMajorCity['AverageTemperature'] = GlobalLandTemperaturesByMajorCity.AverageTemperature.apply(to_fahrenheit)

GlobalLandTemperaturesByMajorCity.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,80.0672,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,81.3812,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,82.5818,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,79.0520,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,77.7686,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [ ]:
#Change Dataframe temperature from Celsius to Fahrenheit
temp_columns = ['LandAverageTemperature',
                'LandAverageTemperatureUncertainty',
                'LandMaxTemperature',
                'LandMaxTemperatureUncertainty',
                'LandMinTemperature',
                'LandMinTemperatureUncertainty',
                'LandAndOceanAverageTemperature',
                'LandAndOceanAverageTemperatureUncertainty'
                ]
GlobalTemperatures[temp_columns] = GlobalTemperatures[temp_columns].apply(to_fahrenheit)
GlobalTemperatures.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,37.4612,38.4332,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,37.5494,38.6636,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,42.1268,37.5368,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,47.2820,36.4118,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,52.8314,35.7296,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Widgets to select the type of temperature to display
stylesheet = """.bk-input-group label {
                  font-size: 20px;
                  font-weight: bold;
                }
             """

temp_select = pn.widgets.Select(name='Temperature',
                                options=['AverageLandTemperature',
                                         'AverageLandAndOceanTemperature',
                                         'AverageLandMaxTemperature',
                                         'AverageLandMinTemperature'
                                         ],
                                value = 'AverageLandTemperature',
                                stylesheets=[stylesheet],
                                )

temp_select.margin = (30, 0, 10, 60)
temp_select

Select(margin=(30, 0, 10, 60), name='Temperature', options=['AverageLandTemperature',...], stylesheets=['.bk-input-group l...], value='AverageLandTemperature')

In [ ]:
#Button to select display between months or seasons

stylesheet = '''.bk-btn-group.bk-horizontal button {
                          font-size: 16px;
                          font-weight: bold;
}'''

seasonal = pn.widgets.RadioButtonGroup(
    name='Seasons / Month', options={'Seasons': 'season', 'Monthly': 'month'},
    button_type='default',
    value = 'season',
    button_style = 'solid',
    stylesheets=[stylesheet]
    )
seasonal.margin = (30, 0, 10, 60)

seasonal

RadioButtonGroup(margin=(30, 0, 10, 60), name='Seasons / Month', options={'Seasons': 'season', ...}, stylesheets=['.bk-btn-group.bk-horizon...], value='season')

In [ ]:
#Year Slider
styles =  {
    'font-size': '18px',
    'font-weight': 'bold'
}

year_slider = pn.widgets.IntSlider(name='Year', start=1850, end=2000, step=25, value=2000,  styles=styles)
year_slider.margin = (20, 0, 10, 60)
year_slider

IntSlider(end=2000, margin=(20, 0, 10, 60), name='Year', start=1850, step=25, styles={'font-size': '18px', ...}, value=2000)

In [ ]:
#Save Button
stylesheet = '''.bk-btn-primary {
  font-size: 14px;
  padding: 10px 20px;
}'''

save_button = pnw.Button(name="Save as .html", button_type="primary", button_style = 'solid', stylesheets=[stylesheet])
save_button.margin = (30, 0, 0, 60)
save_button

Button(button_type='primary', margin=(30, 0, 0, 60), name='Save as .html', stylesheets=['.bk-btn-primary {...])

In [ ]:
#Dashboard Description
description = pn.pane.Markdown("""
#### As greenhouse gases blanket the Earth, they trap the sun's heat.
#### This leads to global warming and climate change. The world is now
#### warming faster than at any point in recorded history. This dashboard
#### explores temperature changesover time and dissects the relationship
#### between temperatures, regions, and seasons.""")
description

Markdown(str)

In [ ]:
#Get Year from dt then rename column for plot
GlobalTemperatures['dt'] = pd.to_datetime(GlobalTemperatures['dt'])
GlobalTemperatures['year'] = GlobalTemperatures['dt'].dt.year
YearTemperatures = GlobalTemperatures.groupby('year')[['LandAverageTemperature', 'LandMaxTemperature', 'LandMinTemperature', 'LandAndOceanAverageTemperature']].mean().reset_index()
YearTemperatures = YearTemperatures.rename(columns={
    'LandAverageTemperature': 'AverageLandTemperature',
    'LandMinTemperature': 'AverageLandMinTemperature',
    'LandMaxTemperature': 'AverageLandMaxTemperature',
    'LandAndOceanAverageTemperature': 'AverageLandAndOceanTemperature'
})
YearTemperatures.head()

,year,AverageLandTemperature,AverageLandMaxTemperature,AverageLandMinTemperature,AverageLandAndOceanTemperature
0,1750,47.694855,NaN,NaN,NaN
1,1751,46.357057,NaN,NaN,NaN
2,1752,42.403700,NaN,NaN,NaN
3,1753,47.098550,NaN,NaN,NaN
4,1754,47.244800,NaN,NaN,NaN


In [ ]:
#Average Temperature scatter plot by year

@pn.depends(temp_select.param.value)
def temp_plot(temp_value):
  plot = px.scatter(YearTemperatures,
                    x = 'year',
                    y= temp_value,
                    trendline="ols",
                    width=700,
                    height=750,
                    title = temp_value
                    )
  plot.update_layout(margin={"r":0, "t":30},
                     title_x = 0.55)
  return plot

pn.Row(temp_select, temp_plot)

Row
    [0] Select(margin=(30, 0, 10, 60), name='Temperature', options=['AverageLandTemperature',...], stylesheets=['.bk-input-group l...], value='AverageLandTemperature')
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)

In [ ]:
#Get Season & Rename Columns

GlobalTemperatures['year'] = GlobalTemperatures['dt'].dt.year
GlobalTemperatures['month'] = GlobalTemperatures['dt'].dt.month_name()
GlobalTemperatures
def season(x):
  if x in ['March', 'April', 'May']:
    return 'Spring'
  elif x in ['June', 'July', 'August']:
    return 'Summer'
  elif x in ['September', 'October', 'November']:
    return 'Fall'
  elif x in ['December', 'January', 'February']:
    return 'Winter'

seasons = GlobalTemperatures['month'].map(season)
GlobalTemperatures['season'] = seasons

GlobalTemperatures1 = GlobalTemperatures.rename(columns={
    'LandAverageTemperature': 'AverageLandTemperature',
    'LandMinTemperature': 'AverageLandMinTemperature',
    'LandMaxTemperature': 'AverageLandMaxTemperature',
    'LandAndOceanAverageTemperature': 'AverageLandAndOceanTemperature'
})

GlobalTemperatures1.head()

,dt,AverageLandTemperature,LandAverageTemperatureUncertainty,AverageLandMaxTemperature,LandMaxTemperatureUncertainty,AverageLandMinTemperature,LandMinTemperatureUncertainty,AverageLandAndOceanTemperature,LandAndOceanAverageTemperatureUncertainty,year,month,season
0,1750-01-01,37.4612,38.4332,NaN,NaN,NaN,NaN,NaN,NaN,1750,January,Winter
1,1750-02-01,37.5494,38.6636,NaN,NaN,NaN,NaN,NaN,NaN,1750,February,Winter
2,1750-03-01,42.1268,37.5368,NaN,NaN,NaN,NaN,NaN,NaN,1750,March,Spring
3,1750-04-01,47.2820,36.4118,NaN,NaN,NaN,NaN,NaN,NaN,1750,April,Spring
4,1750-05-01,52.8314,35.7296,NaN,NaN,NaN,NaN,NaN,NaN,1750,May,Spring


In [ ]:
seasons_color = {'Fall': 'gold',
                 'Summer':'orangered',
                 'Spring':'lightgreen',
                 'Winter':'lightskyblue'}
month_colors = {'January': 'blue',
                'February': 'green',
                'March': 'red',
                'April': 'purple',
                'May': 'orange',
                'June': 'cyan',
                'July': 'pink',
                'August': 'brown',
                'September': 'yellow',
                'October': 'lime',
                'November': 'indigo',
                'December': 'gray'}

#Average temperature by month & seasons plot

@pn.depends(temp_select.param.value, seasonal.param.value)
def seasons_month_plot(temp_value, seasonal_value):
  filtered = GlobalTemperatures1.groupby(by=[seasonal_value, 'year'])[['AverageLandTemperature', 'AverageLandMaxTemperature', 'AverageLandMinTemperature', 'AverageLandAndOceanTemperature']].mean().reset_index()
  plot = px.scatter(
      filtered,
      x = 'year',
      y= temp_value,
      color = seasonal.value,
      color_discrete_map = seasons_color if seasonal_value == 'season' else month_colors,
      trendline="ols",
      opacity = 1 if seasonal_value == 'season' else .2,
      width=700,
      height=750,
      title = temp_value +' by ' + seasonal_value
      );

  plot.update_layout(margin={"t":30},
                     title_x = 0.48)
  return plot;

pn.Row(seasonal, temp_select, seasons_month_plot)

Row
    [0] RadioButtonGroup(margin=(30, 0, 10, 60), name='Seasons / Month', options={'Seasons': 'season', ...}, stylesheets=['.bk-btn-group.bk-horizon...], value='season')
    [1] Select(margin=(30, 0, 10, 60), name='Temperature', options=['AverageLandTemperature',...], stylesheets=['.bk-input-group l...], value='AverageLandTemperature')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False)

In [ ]:
GlobalLandTemperaturesByMajorCity.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,80.0672,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,81.3812,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,82.5818,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,79.0520,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,77.7686,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [ ]:
#Covert Coordinates from East/West/North/South to floats

def convert(coordinate):
  degrees = float(coordinate[:-1])
  direction = coordinate[-1]
  if direction in ['W', 'S']:
    degrees *= -1
  return degrees

Latitude = GlobalLandTemperaturesByMajorCity.Latitude.map(convert)
Longitude = GlobalLandTemperaturesByMajorCity.Longitude.map(convert)
GlobalLandTemperaturesByMajorCity['Latitude'] = Latitude
GlobalLandTemperaturesByMajorCity['Longitude'] = Longitude

GlobalLandTemperaturesByMajorCity['dt'] = pd.to_datetime(GlobalLandTemperaturesByMajorCity['dt'], format='%Y-%m-%d', errors='coerce')
GlobalLandTemperaturesByMajorCity['year'] = GlobalLandTemperaturesByMajorCity['dt'].dt.year

GlobalLandTemperaturesByMajorCity

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year
0,1849-01-01,80.0672,1.435,Abidjan,Côte D'Ivoire,5.63,-3.23,1849
1,1849-02-01,81.3812,1.362,Abidjan,Côte D'Ivoire,5.63,-3.23,1849
2,1849-03-01,82.5818,1.612,Abidjan,Côte D'Ivoire,5.63,-3.23,1849
3,1849-04-01,79.0520,1.387,Abidjan,Côte D'Ivoire,5.63,-3.23,1849
4,1849-05-01,77.7686,1.200,Abidjan,Côte D'Ivoire,5.63,-3.23,1849
...,...,...,...,...,...,...,...,...
239171,2013-04-01,54.6134,1.823,Xian,China,34.56,108.97,2013
239172,2013-05-01,66.1622,0.807,Xian,China,34.56,108.97,2013
239173,2013-06-01,74.3396,0.647,Xian,China,34.56,108.97,2013
239174,2013-07-01,77.4518,1.042,Xian,China,34.56,108.97,2013


In [ ]:
#Heatmap Based on Longitude & Latitude by Year

@pn.depends(year_slider.param.value)
def heatmap(year):
  filtered = GlobalLandTemperaturesByMajorCity[GlobalLandTemperaturesByMajorCity['year'] == year]
  grouped = filtered.groupby(by=['Longitude', 'Latitude'])['AverageTemperature'].mean().reset_index()
  plot = px.density_heatmap(grouped,
                            x = 'Longitude',
                            y = 'Latitude',
                            z = 'AverageTemperature',
                            histfunc="avg",
                            width=550,
                            height=400,
                            color_continuous_scale='bluered',
                            text_auto=True,
                            title = 'Year ' + str(year) + ' Avg Temp by Coordinates'
                            )
  plot.update_layout(coloraxis_colorbar_title='Avg Temp',
                     coloraxis_colorbar=dict(title_font=dict(size=12), x = 1),
                     title_x = 0.5,
                     title_y = 0.9,
                     margin={"b":0, 'l':0, 't': 80})
  return plot
pn.Row(year_slider, heatmap)

Row
    [0] IntSlider(end=2000, margin=(20, 0, 10, 60), name='Year', start=1850, step=25, styles={'font-size': '18px', ...}, value=2000)
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)

In [ ]:
GlobalLandTemperaturesByCountry.head()
GlobalLandTemperaturesByCountry['dt'] = pd.to_datetime(GlobalLandTemperaturesByCountry['dt'], format='%Y-%m-%d', errors='coerce')
GlobalLandTemperaturesByCountry['year'] = GlobalLandTemperaturesByCountry['dt'].dt.year
GlobalLandTemperaturesByCountry.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,year
0,1743-11-01,39.8912,36.1292,Åland,1743
5,1744-04-01,34.7540,40.4240,Åland,1744
6,1744-05-01,44.0636,35.2202,Åland,1744
7,1744-06-01,52.8962,34.8386,Åland,1744
8,1744-07-01,59.6156,34.5380,Åland,1744


In [ ]:
#World Temperature Choropleth Map

@pn.depends(year_slider.param.value)
def country_map(year):
  filtered = GlobalLandTemperaturesByCountry[GlobalLandTemperaturesByCountry['year'] == year]
  grouped = filtered.groupby(by=['Country'])['AverageTemperature'].mean().reset_index()
  country_map = px.choropleth(grouped,
                            locationmode = 'country names',
                            locations = 'Country',
                            color = 'AverageTemperature',
                            color_continuous_scale='spectral_r'
                            )
  country_map.update_layout(margin={"r":0,"t":80,"l":0,"b":0},
                            height = 400,
                            width = 850,
                            title_text = 'Year ' + str(year) + ' World Temperature Map',
                            coloraxis_colorbar_title='Avg Temp',
                            coloraxis_colorbar=dict(title_font=dict(size=12), x = 0.88),
                            title_y = 0.9,
                            title_x = 0.5
                            )
  return country_map
pn.Row(year_slider, country_map)

Row
    [0] IntSlider(end=2000, margin=(20, 0, 10, 60), name='Year', start=1850, step=25, styles={'font-size': '18px', ...}, value=2000)
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)

In [ ]:
#Globe Image for Dashboard

globe_pane = pn.pane.Image('https://upload.wikimedia.org/wikipedia/commons/2/22/Earth_Western_Hemisphere_transparent_background.png', width = 400)
globe_pane.margin = (20, 0, 20, 40)

globe_pane

Image(str, margin=(20, 0, 20, 40), width=400)

In [ ]:
#Dashboard
row1 = pn.Row(country_map, heatmap)
row2=  pn.Row(temp_plot, seasons_month_plot)
column1 = pn.Column('# World Temperature Dashboard', globe_pane, description, '## Settings', year_slider, temp_select, seasonal, save_button)
column2 = pn.Column(row1, row2)
dashboard = pn.Row(column1, column2)

#Save Function Button
def save_dashboard(event):
  dashboard.save(filename="WorldTemperatureDashboard.html" , title="World Temperature Dashboard")

save_button.on_click(save_dashboard)

dashboard
#dashboard.save(filename = 'dashboard.html', embed = True)

Row
    [0] Column
        [0] Markdown(str)
        [1] Image(str, margin=(20, 0, 20, 40), width=400)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] IntSlider(end=2000, margin=(20, 0, 10, 60), name='Year', start=1850, step=25, styles={'font-size': '18px', ...}, value=2000)
        [5] Select(margin=(30, 0, 10, 60), name='Temperature', options=['AverageLandTemperature',...], stylesheets=['.bk-input-group l...], value='AverageLandTemperature')
        [6] RadioButtonGroup(margin=(30, 0, 10, 60), name='Seasons / Month', options={'Seasons': 'season', ...}, stylesheets=['.bk-btn-group.bk-horizon...], value='season')
        [7] Button(button_type='primary', margin=(30, 0, 0, 60), name='Save as .html', stylesheets=['.bk-btn-primary {...])
    [1] Column
        [0] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
            [1] ParamFunction(function, _pane=Plotly, defer_load=False)
        [1] Row
            [0] ParamFunction(function, _pane=Plotly, defer_load=False)
            [1] ParamFunction(function, _pane=Plotly, defer_load=False)